# LIBRARY

In [19]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk import tokenize
from b2_preprocessing_function import CustomPreprocess

ModuleNotFoundError: No module named 'b2_preprocessing_function'

In [3]:
df=pd.read_excel('D:\Python\PyProject\Học máy\Data1.xlsx')
df.head()

,reviewer_name,reviewer_country,rental_date,comments,langugage
0,Igor,"Kraków, Ba Lan",2023-12-02 00:00:00,The best support I have ever witnessed. Answer...,en
1,Kevin,"Cannes, Pháp",2023-12-04 00:00:00,Everything is very ok.,en
2,Walid,"Cannes, Pháp",2023-12-09 00:00:00,"Great accommodation for some games, the owner ...",en
3,Maxence,Vương quốc Anh,2023-12-11 00:00:00,Thanks Nigel & Tra for friendly stay in Ho Chi...,en
4,Lisa,Ai/len,2023-12-10 00:00:00,Nigel and others were very dedicated and respo...,en


In [10]:
number_countries = df['reviewer_country'].nunique()
print(number_countries)

377


In [13]:
number_reviewer=df['reviewer_name'].nunique()
print(number_reviewer)

106678


In [18]:
stopwords_list = set(stopwords.words('english'))


TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    '''Removes HTML tags: replaces anything between opening and closing <> with empty space'''

    return TAG_RE.sub('', text)
    

class CustomPreprocess():
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''

    def __init__(self):
        pass

    def preprocess_text(self,sen):
        sen = sen.lower()
        
        # Remove html tags
        sentence = remove_tags(sen)

        # Remove punctuations and numbers
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        
        # Single character removal
        sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

        # Remove multiple spaces
        sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.
        
        # Remove Stopwords
        pattern = re.compile(r'\b(' + r'|'.join(stopwords_list) + r')\b\s*')
        sentence = pattern.sub('', sentence)
        
        return sentence


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\hkhiy/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2288.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\hkhiy\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
X = []
sentences = list(movie_reviews['review'])
for sen in sentences:
    X.append(custom.preprocess_text(sen))